In [1]:
import numpy as np
import pandas as pd
import pgeocode as pg
from utils import listdir_nohidden, assign_types
import os

nomi = pg.Nominatim('gb')

path = 'raw_ip/'

month_folders = listdir_nohidden(path) # path to downloaded data

address = address = pd.read_csv(path+'IPData202301'+'/IPAddress.csv',
                                usecols=['OCSCode', 'PostCode'])

zips = address['PostCode'].apply(lambda x: x[:4]) # get 1st 4 chars of postcode
geo_query = nomi.query_postal_code(zips.to_numpy()) # lookup coords from postcodes

# Add coords
address['lat'] = geo_query['latitude']
address['long'] = geo_query['longitude']

address

,OCSCode,PostCode,lat,long
0,P00002,HR3 5AE,52.087542,-3.049270
1,P00016,LL29 8DG,53.283275,-3.696600
2,P00017,LL21 0DE,52.975586,-3.389057
3,P00018,CH5 1BX,53.199300,-3.036140
4,P00029,CH7 1AP,53.172097,-3.163118
...,...,...,...,...
183,6A2,LL55 1HU,53.129154,-4.195054
184,6B4,LL13 7TD,52.991762,-2.911681
185,6A6,SA1 5AW,51.633980,-3.948480
186,6A5,SA10 6JQ,51.714218,-3.720373


In [2]:
ip_datas = []
for month in month_folders:
    ip_path = [f for f in os.listdir(path+month) if f.startswith("IPData")][0]
    ip_data = pd.read_csv(path+month+'/'+ip_path, 
                          usecols=['Period', 'BNFCode', 'PracticeID', 'Quantity', 'DDD'])
    
    ip_data['Quantity/DDD'] = ip_data['Quantity']/ip_data['DDD']

    ip_data = assign_types(ip_data)
    
    ip_datas.append(ip_data)

ip_datas = pd.concat(ip_datas, axis=0)

ip_datas

,PracticeID,BNFCode,Quantity,DDD,Period,Quantity/DDD,type
0,P00226,0501012G0AAABAB,84,21.0,202011,4.0,penicillins
1,P00226,0501013B0AAABAB,21,7.0,202011,3.0,penicillins
2,P00226,0501013K0AAAJAJ,21,7.0,202011,3.0,penicillins
7,P00239,0501012G0AAABAB,28,7.0,202011,4.0,penicillins
11,P00241,0501030I0AAABAB,6,6.0,202011,1.0,tetracyclines
...,...,...,...,...,...,...,...
678,P00406,0501050B0AAADAD,10,10.0,202202,1.0,macrolides
725,P00296,0501011P0AAAJAJ,40,5.0,202202,8.0,penicillins
726,P00296,0501012G0AAABAB,28,7.0,202202,4.0,penicillins
727,P00296,0501013B0AAABAB,21,7.0,202202,3.0,penicillins


In [4]:
merged = pd.merge(ip_datas, address, 
                  left_on='PracticeID',
                  right_on='OCSCode',
                  how='left').drop(columns='OCSCode')

In [6]:
print(merged['Quantity/DDD'].isnull().sum())

merged

0


,PracticeID,BNFCode,Quantity,DDD,Period,Quantity/DDD,type,PostCode,lat,long
0,P00226,0501012G0AAABAB,84,21.0,202011,4.0,penicillins,NP22 3NG,51.772500,-3.246800
1,P00226,0501013B0AAABAB,21,7.0,202011,3.0,penicillins,NP22 3NG,51.772500,-3.246800
2,P00226,0501013K0AAAJAJ,21,7.0,202011,3.0,penicillins,NP22 3NG,51.772500,-3.246800
3,P00239,0501012G0AAABAB,28,7.0,202011,4.0,penicillins,CF81 8QS,51.710483,-3.265000
4,P00241,0501030I0AAABAB,6,6.0,202011,1.0,tetracyclines,CF81 8NX,51.710483,-3.265000
...,...,...,...,...,...,...,...,...,...,...
4539,P00406,0501050B0AAADAD,10,10.0,202202,1.0,macrolides,SA11 5HP,51.708007,-3.694407
4540,P00296,0501011P0AAAJAJ,40,5.0,202202,8.0,penicillins,CF41 7PE,51.654300,-3.491300
4541,P00296,0501012G0AAABAB,28,7.0,202202,4.0,penicillins,CF41 7PE,51.654300,-3.491300
4542,P00296,0501013B0AAABAB,21,7.0,202202,3.0,penicillins,CF41 7PE,51.654300,-3.491300


In [ ]:
merged.to_csv('FilteredIPData.csv.zip', index=False, compression='gzip')